In [2]:
import pandas as pd

In [6]:
# #Prepare daily timeseries data
# import yfinance as yf
# import datetime

symbols = ['1333.T', '1332.T', '1605.T']
# start_date = datetime.datetime.now() - datetime.timedelta(days=360)
# end_date = datetime.datetime.now()

# df = yf.download(symbols, group_by="ticker", interval="1d", start=start_date, end=end_date)
# df.to_csv("./yfinance_1d_ex.csv")

[*********************100%***********************]  3 of 3 completed


In [3]:
df = pd.read_csv("../SampleData/yfinance_1d_ex.csv" ,header=[0,1], index_col=0, parse_dates=True)
df.iloc[:10]

1605.T                                               1332.T  \
              Open    High     Low   Close   Adj Close    Volume   Open   
Date                                                                      
2021-12-16   996.0  1013.0   996.0  1011.0  964.446960  10983400  543.0   
2021-12-17  1020.0  1026.0  1011.0  1019.0  972.078552  11581400  536.0   
2021-12-20  1000.0  1003.0   969.0   971.0  926.288757  13200800  530.0   
2021-12-21   985.0   989.0   976.0   980.0  934.874390   7457100  544.0   
2021-12-22   992.0   999.0   990.0   995.0  949.183716   7489600  541.0   
2021-12-23  1009.0  1030.0  1009.0  1020.0  973.032532  11184300  544.0   
2021-12-24  1025.0  1031.0  1009.0  1013.0  966.354858   6404400  543.0   
2021-12-27  1015.0  1016.0   998.0  1010.0  963.492981   5405900  541.0   
2021-12-28  1014.0  1026.0  1014.0  1020.0  973.032532  10611900  541.0   
2021-12-29  1015.0  1016.0   996.0  1007.0  987.745728   6620700  545.0   

                                                      1333.T                  \
             High    Low  Close   Adj Close   Volume    Open    High     Low   
Date                                                                           
2021-12-16  544.0  535.0  539.0  523.639648  1542700  2445.0  2445.0  2423.0   
2021-12-17  542.0  533.0  534.0  518.782104  1834100  2437.0  2441.0  2400.0   
2021-12-20  536.0  529.0  532.0  516.839111  1361000  2370.0  2391.0  2359.0   
2021-12-21  546.0  537.0  538.0  522.668152  1507600  2388.0  2388.0  2365.0   
2021-12-22  543.0  539.0  539.0  523.639648   968500  2378.0  2382.0  2360.0   
2021-12-23  549.0  542.0  544.0  528.497131  1447400  2379.0  2386.0  2366.0   
2021-12-24  545.0  540.0  543.0  527.525635   821100  2366.0  2374.0  2356.0   
2021-12-27  543.0  537.0  537.0  521.696594   912600  2359.0  2361.0  2339.0   
2021-12-28  549.0  539.0  549.0  533.354675  1376200  2355.0  2400.0  2353.0   
2021-12-29  553.0  545.0  549.0  533.354675  1379900  2403.0  2425.0  2403.0   

                                         
             Close    Adj Close  Volume  
Date                                     
2021-12-16  2437.0  2384.313232   93400  
2021-12-17  2401.0  2349.091553  135100  
2021-12-20  2365.0  2313.869873   90800  
2021-12-21  2376.0  2324.632080   75100  
2021-12-22  2368.0  2316.804932   55400  
2021-12-23  2371.0  2319.740234   68000  
2021-12-24  2356.0  2305.064453   66600  
2021-12-27  2352.0  2301.150879   81900  
2021-12-28  2400.0  2348.113281  138200  
2021-12-29  2418.0  2365.724121   80700

In [8]:
def concat_df_symbols(org_dfs, dfs, symbols:list, column_name:str, grouped_by_symbol=False):
    if grouped_by_symbol:
        df_cp = org_dfs.copy()
        dfs_cp = dfs.copy()
        dfs_cp.columns = symbols
        for symbol in symbols:
            df_cp[(symbol, column_name)] = dfs_cp[symbol]
        return df_cp
    else:
        #dfs.columns = pd.MultiIndex.from_tuples([(column_name, symbol) for symbol in symbols])
        dfs.columns = [(column_name, symbol) for symbol in symbols]
        return pd.concat([org_dfs, dfs], axis=1)

def get_symbols(dfs:pd.DataFrame, grouped_by_symbol=False):
    if type(dfs.columns) == pd.MultiIndex:
        if grouped_by_symbol:
            symbols = list(set(dfs.columns.droplevel(1)))
            return symbols
        else:
            column = dfs.columns[0][0]
            return list(dfs[column].columns)
        
def create_multi_out_lists(symbols, elements, columns, grouped_by_symbol=False):
    out_columns = []
    out_eles = []
        
    for index in range(0, len(columns)):
        if columns[index]:
            column = columns[index]
            out_eles.append(elements[index])
            out_columns.extend(((column, _symbol) for _symbol in symbols))
    out_index = pd.MultiIndex.from_tuples(out_columns)
    if grouped_by_symbol:
        out_index = out_index.swaplevel(0, 1)
    return out_eles, out_index

## EMA

In [3]:
def EMA(df, interval):
    return df.ewm(span=interval, adjust=False).mean()

## ATR

In [4]:
def ATRFromOHLC(data: pd.DataFrame, ohlc_columns = ('Open', 'High', 'Low', 'Close'), window = 14,
                tr_name="TR", atr_name="ATR"):
    """
    function to calculate True Range and Average True Range
    
    Args:
        data (pd.DataFrame): ohlc data
        ohlc_columns (tuple, optional): Defaults to ('Open', 'High', 'Low', 'Close').
        window (int, optional): Defaults to 14.

    Returns:
        pd.Series: Name:ATR, dtype:float64. inlucdes Null till window size
    """
    high_cn = ohlc_columns[1]
    low_cn = ohlc_columns[2]
    close_cn = ohlc_columns[3]
    
    df = data.copy()
    df["H-L"] = df[high_cn] - df[low_cn]
    df["H-PC"] = abs(df[high_cn] - df[close_cn].shift(1))
    df["L-PC"] = abs(df[low_cn] - df[close_cn].shift(1))
    df[tr_name] = df[["H-L","H-PC","L-PC"]].max(axis=1)
    df[atr_name] = EMA(df[tr_name], interval=window)
    
    return df[[tr_name, atr_name]].copy()

In [9]:
def ATRFromMultiOHLC(symbols:list, data: pd.DataFrame, ohlc_columns = ('Open', 'High', 'Low', 'Close'), window = 14, grouped_by_symbol=False,
                     tr_name="TR", atr_name="ATR"):
    high_cn = ohlc_columns[1]
    low_cn = ohlc_columns[2]
    close_cn = ohlc_columns[3]
        
    df = data.copy()
    if grouped_by_symbol:
        df.columns = df.columns.swaplevel(0, 1)
    
    df = df[[(column, symbol) for symbol in symbols for column in ohlc_columns]]
        
    hl_df = df[high_cn] - df[low_cn]
    hpc_df = abs(df[high_cn] - df[close_cn].shift(1))
    lpc_df = abs(df[low_cn] - df[close_cn].shift(1))
    
    temp_df = pd.concat([hl_df, hpc_df, lpc_df], axis=1)
    tr_df = pd.DataFrame(index=temp_df.index)
    for symbol in symbols:
        tr_df[symbol] = temp_df[symbol].max(axis=1)
    atr_df = EMA(tr_df, window)
    elements, columns = create_multi_out_lists(symbols, [tr_df, atr_df], [tr_name, atr_name], grouped_by_symbol)
    out_df = pd.concat(elements, axis=1)
    out_df.columns = columns
    out_df.sort_index(level=0, axis=1, inplace=True)
    return out_df

## Renko

In [38]:
def RenkoFromSeries(data_sr:pd.Series, brick_size,
                    total_brick_name = "Renko", brick_num_name = "Brick"):
    """ Caliculate brick number of Renko

    Args:
        data_sr (pd.Series): time series data like close values of a sygnal
        brick_size (pd.Series|float, optional): brick_size to caliculate the Renko. If None, ATR is used. Defaults to None.
        
    Returns:
        pd.Series: brick_num
    """
    def get_check_df_from_series(data_sr, brick_sr, start_index, to_index, criteria_value):
        return (data_sr.iloc[start_index: to_index] - criteria_value)/brick_sr.iloc[start_index: to_index]
    
    def get_check_df_from_scalar(data_sr, brick_size, start_index, to_index, criteria_value):
        return (data_sr.iloc[start_index: to_index] - criteria_value)/brick_size
    
    if type(brick_size) == pd.Series:
        if len(data_sr) != len(brick_size):
            raise Exception(f"sr and brick_size_sr should have same length.")
        brick_size = brick_size.copy().reset_index(drop=True)
        get_check_df = get_check_df_from_series
    else:
        get_check_df = get_check_df_from_scalar
            
    org_index = data_sr.index
    sr = data_sr.copy().reset_index(drop=True)
    
    def trendy(uptrend, downtrend):
        if len(uptrend) > 0 and len(downtrend) > 0:
            if uptrend.index[0] > downtrend.index[0]:
                #mark down until criteria_index to downtrend.index[0]
                trend = -1
                brick_size = int(downtrend.iloc[0])
                next_criteria_index = downtrend.index[0]
            else:
                #mark up until criteria_index to uptrend.index[0]
                trend = 1
                brick_size = int(uptrend.iloc[0])
                next_criteria_index = uptrend.index[0]
        elif len(uptrend) > 0:
            trend = 1
            brick_size = int(uptrend.iloc[0])
            next_criteria_index = uptrend.index[0]
        elif len(downtrend) > 0:
            trend = -1
            brick_size = int(downtrend.iloc[0])
            next_criteria_index = downtrend.index[0]
        else:
            trend = None
            brick_size = None
            next_criteria_index = None
        return trend, brick_size, next_criteria_index
    
    CONST_INDEX_PLUS = 30
    total_brick_num_sr = pd.Series(0, index=sr.index)
    brick_value_sr = pd.Series(0, index=sr.index)
    
    try:
        criteria_index = sr[pd.notna(sr)].index[0]
    except IndexError:
        print(f"can't criteria index with {len(pd.notna(sr))} of {len(sr)}")
        renko_df = pd.DataFrame.from_dict({total_brick_name:total_brick_num_sr, brick_num_name:brick_value_sr})
        renko_df.index = org_index
        return renko_df
    current_criteria = sr.iloc[criteria_index]

    trend = None
    start_index = criteria_index
    to_index = criteria_index + CONST_INDEX_PLUS
    while trend is None and to_index < len(sr):
        temp_brick_num_sr = get_check_df(sr, brick_size, start_index, to_index, current_criteria)
        brick_value_sr.iloc[start_index:to_index] = temp_brick_num_sr
        uptrend = temp_brick_num_sr[temp_brick_num_sr >= 1]
        downtrend = temp_brick_num_sr[temp_brick_num_sr <= -1]
        #if trend is None:
        if len(uptrend) == 0 and len(downtrend) == 0:
            start_index = to_index
            to_index = start_index + CONST_INDEX_PLUS
        else:
            trend, block_num, next_criteria_index = trendy(uptrend, downtrend)
    if trend is None:
        renko_df = pd.DataFrame.from_dict({total_brick_name:total_brick_num_sr, brick_num_name:brick_value_sr})
        renko_df.index = org_index
        return renko_df
    
    global_trend = trend
    pre_trend = trend
    while True:
        trend, new_brick_num, next_criteria_index = trendy(uptrend, downtrend)        
        if trend is None:#didn't changed renko value
            brick_num = total_brick_num_sr.iloc[criteria_index]
            total_brick_num_sr.iloc[criteria_index:to_index] = brick_num
            next_criteria_index = criteria_index
            next_start_index = to_index
        else:
            global_trend = trend
            brick_num = total_brick_num_sr.iloc[criteria_index]
            if pre_trend/trend >= 0:#continuaus trend
                next_brick_num = brick_num + new_brick_num
            else:
                next_brick_num = brick_num + new_brick_num - trend

            total_brick_num_sr.iloc[criteria_index: next_criteria_index] = brick_num
            total_brick_num_sr.iloc[next_criteria_index] = next_brick_num
            criteria_index = next_criteria_index
            next_start_index = next_criteria_index + 1
            pre_trend = trend
            
        if next_start_index < len(sr):
            to_index = next_start_index + CONST_INDEX_PLUS
            if to_index > len(sr):
                to_index = len(sr)

            current_criteria = sr.iloc[next_criteria_index]
            temp_brick_num_sr = get_check_df(sr, brick_size, next_start_index, to_index, current_criteria)
            brick_value_sr.iloc[next_start_index:to_index] = temp_brick_num_sr
            uptrend = temp_brick_num_sr[temp_brick_num_sr >=  -global_trend/2 + 3/2]
            downtrend = temp_brick_num_sr[temp_brick_num_sr <= -global_trend/2 - 3/2]
        else:
            break
    #total_brick_num_sr.index = org_index
    renko_df = pd.DataFrame.from_dict({total_brick_name:total_brick_num_sr, brick_num_name:brick_value_sr})
    renko_df.index = org_index
    return renko_df

In [34]:
def RenkoFromOHLC(df:pd.DataFrame, ohlc_columns = ('Open', 'High', 'Low', 'Close'), brick_column_name=None, brick_size=None, atr_window=14,
                  total_brick_name = "Renko", brick_num_name = "Brick"):
    """ Caliculate Renko from Close column of ohlc dataframe

    Args:
        df (pd.DataFrame): time series data of Open, High, Low and Close
        ohlc_columns (tuple, optional): columns names of OHLC. Defaults to ('Open', 'High', 'Low', 'Close').
        brick_column_name (str, optional): column name of brick_size. Defaults to None.
        brick_size (pd.Series|float, optional): brick_size to caliculate the Renko. If None, ATR is used. Defaults to None.

    Raises:
        Exception: When ohlc_columns is not a subset of df.columns

    Returns:
        pd.DataFrame: brick numbers are stored on brick_num column
    """
    if(set(ohlc_columns).issubset(df.columns)):
        if brick_size is None:
            if brick_column_name is None:
                atr_df = ATRFromOHLC(df, ohlc_columns, window=atr_window, atr_name="ATR")
                brick_size = atr_df["ATR"]
            else:
                brick_size = df[brick_column_name]
            renko_df = RenkoFromSeries(df[ohlc_columns[3]], brick_size=brick_size, total_brick_name=total_brick_name, brick_num_name=brick_num_name)
        else:
            renko_df = RenkoFromSeries(df[ohlc_columns[3]], brick_size=brick_size, total_brick_name=total_brick_name, brick_num_name=brick_num_name)
        return renko_df
    else:
        raise Exception(f"specified ohlc_columns {ohlc_columns} doen't match with df.columns {df.columns}")

In [39]:
atr_dfs = ATRFromMultiOHLC(symbols, df.iloc[:100], ["Open", "High", "Low", "Close"], window=14, grouped_by_symbol=True, atr_name="ATR")
brick_sizes = atr_dfs[[(symbol, "ATR") for symbol in symbols]]
print(brick_sizes)

               1333.T     1332.T     1605.T
                  ATR        ATR        ATR
Date                                       
2021-12-16  22.000000   9.000000  17.000000
2021-12-17  24.533333   9.000000  16.733333
2021-12-20  26.862222   8.733333  21.168889
2021-12-21  26.347259   9.435556  20.746370
2021-12-22  25.767625   8.844148  20.513521
...               ...        ...        ...
2022-05-12  45.386970  16.945856  67.158056
2022-05-13  48.268707  17.353075  63.536982
2022-05-16  49.432880  16.772665  60.532051
2022-05-17  47.108496  16.669643  65.127777
2022-05-18  47.094030  15.380357  62.310740

[100 rows x 3 columns]


In [43]:
DFS = {}
for symbol in symbols:
    DFS[symbol] = RenkoFromOHLC(df[symbol].iloc[:100], ["Open", "High", "Low", "Close"], brick_size=brick_sizes[symbol]["ATR"])

In [44]:
renko_df = pd.concat(DFS.values(), axis=1, keys=DFS.keys())
renko_df

1333.T           1332.T           1605.T          
            Renko     Brick  Renko     Brick  Renko     Brick
Date                                                         
2021-12-16      0  0.000000      0  0.000000      0  0.000000
2021-12-17     -1 -1.467391      0 -0.555556      0  0.478088
2021-12-20     -2 -1.340172      0 -0.801527     -1 -1.889565
2021-12-21     -2  0.417501      0 -0.105982     -1  0.433811
2021-12-22     -2  0.116425      0  0.000000     -1  1.169960
...           ...       ...    ...       ...    ...       ...
2022-05-12      1 -0.440655      1 -2.655517      9 -0.670061
2022-05-13      1  0.683673      1  0.864400      9 -0.834160
2022-05-16      1  0.323671      1  0.417346      9 -0.776448
2022-05-17      1  0.445779      1  0.719871      9  0.552760
2022-05-18      1 -0.191107      1  0.455126      9  0.417264

[100 rows x 6 columns]

In [45]:
renko_df.iloc[-50:]

1333.T           1332.T           1605.T          
            Renko     Brick  Renko     Brick  Renko     Brick
Date                                                         
2022-03-03      2  0.287406      0  0.966531      6  0.456888
2022-03-04      2 -0.089104      0  1.077740      6  0.356862
2022-03-07      2 -0.440374      0  0.000000      7  1.730172
2022-03-08      1 -1.742831     -1 -1.182989      7 -1.241043
2022-03-09      1 -0.310866     -1 -0.580229      7 -1.332241
2022-03-10      1  1.262408     -1  1.658509      7 -1.185423
2022-03-11      1  1.567042     -1  1.941753      7 -0.542277
2022-03-14      2  2.373206      1  3.063060      7 -0.528907
2022-03-15      2  0.622955      1  0.370464      7 -1.734573
2022-03-16      2  0.145047      1  0.191859      7 -1.902927
2022-03-17      2 -0.081466      1  0.677893      7 -1.582342
2022-03-18      2 -0.190557      1  0.357533      7 -1.628245
2022-03-22      2  0.249278      1  0.743306      7  0.093392
2022-03-23      2  0.578326      2  1.583755      7  0.476463
2022-03-24      2  0.137730      2 -0.823456      7  0.890107
2022-03-25      2  0.399881      2 -0.232368      7  0.898102
2022-03-28      3  1.159223      2  0.164983      8  1.110541
2022-03-29      3 -0.264398      2 -0.395586      8  0.156767
2022-03-30      2 -2.004725      2 -1.538027      8 -0.629423
2022-03-31      2 -0.857368      1 -2.266433      8 -0.953234
2022-04-01      2 -0.629254      1  0.068404      8 -1.367832
2022-04-04      2 -0.119085      1  0.499890      8 -0.718600
2022-04-05      2 -0.634704      1  0.227227      8 -0.219587
2022-04-06      1 -1.503308      1 -0.751372      8 -0.564624
2022-04-07      1  0.574198      1 -0.621707      8 -1.228422
2022-04-08      1  0.821872      1  0.313674      8 -1.106748
2022-04-11      1  0.528129      1  0.726677      8 -0.590799
2022-04-12      1 -0.604343      1  0.549719      8 -1.062665
2022-04-13      1  0.000000      1  0.864358      8  0.319401
2022-04-14      1  0.394560      1  1.352158      9  1.378811
2022-04-15      1 -0.219827      1  0.655374      9  0.058055
2022-04-18      1 -0.759211      1 -0.089900      9  0.606229
2022-04-19      1 -0.301035      1  0.283724     10  1.688434
2022-04-20      1  0.737448      1  1.922458     10 -0.315575
2022-04-21      1  0.477506      2  2.281304     10 -0.450953
2022-04-22      1  0.026075      2 -0.553041     10 -1.503864
2022-04-25      1 -0.397128      2 -1.068231      9 -2.594357
2022-04-26      1  0.210887      2 -0.644449      9 -0.876035
2022-04-27      0 -1.195876      2 -1.308771      9 -0.765529
2022-04-28      0  1.543883      2 -0.079395      9  0.216869
2022-05-02      0  1.193222      2 -0.239697      9  0.087691
2022-05-06      0  1.721312      2  0.243631      9  1.258984
2022-05-09      0  0.908708      2 -0.241834      9  1.314130
2022-05-10      1  2.203592      2 -0.245542      9 -0.662066
2022-05-11      1 -0.237224      2 -0.568083      9 -0.581008
2022-05-12      1 -0.440655      1 -2.655517      9 -0.670061
2022-05-13      1  0.683673      1  0.864400      9 -0.834160
2022-05-16      1  0.323671      1  0.417346      9 -0.776448
2022-05-17      1  0.445779      1  0.719871      9  0.552760
2022-05-18      1 -0.191107      1  0.455126      9  0.417264